# ICON Data Examples Using pysat.

In [ ]:
# Import packages
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
import pysat
# pysat needs a path to store and load data from

# If this is the first time you are running pysat then we need to 
# tell pysat where to look for data by running the code below.

# If you are not running pysat for the first time then this demo will
# simply use the existing pysat data directory as already set on the system.

# Set data directory if user hasn't already set one
if len(pysat.params['data_dirs']) == 0:
    # Set a directory for pysat to use for data
    pysat.params['data_dirs'] = '~/pysatDemo'
else:
    print('pysat directory has been set previously. Leaving unchanged.')

# Register instruments with pysat. Only needed once per install.
import pysatNASA
pysat.utils.registry.register_by_module(pysatNASA.instruments)

# Note a warning is raised about non-unique timestamps. This warning is expected
# and is not an actual issue here, but it is being tracked by the pysat team.

# ICON IVM Examples

In [ ]:
# instantiate IVM Instrument object
# IVM data is automatically cleaned using instrument flags as it is loaded
# levels of 'clean', 'dusty', 'dirty', and 'none' or None are supported
# this is generally true for all pysat instruments
ivm = pysat.Instrument('icon', 'ivm', inst_id='a', clean_level='clean')

In [ ]:
# Set range of dates for downloading ICON data
start_download_date = dt.datetime(2020, 1, 1)
stop_download_date = dt.datetime(2020, 1, 4)

In [ ]:
# Download data from NASA CDAWeb.

# If you would like additional feedback while pysat performs operations
# uncomment the line below.
# pysat.logger.setLevel("INFO")

# Download data between specific dates.
ivm.download(start_download_date, stop_download_date)

# To get all of the latest files on the server but not on
# the local machine, uncomment line below.
# The version and revision numbers in the filenames are used to identify
# the latest files. This works for all ICON instruments.
# ivm.download_updated_files()

# Resume limited feedback
# pysat.logger.setLevel("WARNING)

## Quick pysat Primer

In [ ]:
# The pysat.Instrument object provides an interface to your file and data needs

# While Instrument objects may be re-used, they are instantiated in each cell
# in this demo to reduce inter-dependence between Jupyter cells
ivm = pysat.Instrument('icon', 'ivm', inst_id='a')

# Load a day of data. All code options below will produce the same result.

# Load by year and day of year
ivm.load(2020, 1)

# Load by date instead
# ivm.load(date=dt.datetime(2020, 1, 1))

# Load by filename instead
# ivm.load(fname=ivm.files[0])
# ivm.load(fname=ivm.files[dt.datetime(2020, 1, 1)])

# All instrument data found at
ivm.data

# For IVM, the data format is a pandas DataFrame while the other ICON
# instruments are all xarray DataSets.

# Metadata information is found at
ivm.meta

# Date of loaded data
print('Loaded data date: ' + ivm.date.strftime('%Y-%m-%d'))
print('')

In [ ]:
# Variables may be set and obtained using strings at the Instrument level
# or by interacting with the underlying data object itself.
# Note that all values are indexed by a datetime.
print('Variable : Ion Density')
print(ivm['Ion_Density'])
# print(ivm.data['Ion_Density'])
print('')

# Slicing and dicing mini-examples
print('Subset of data')
print(ivm[0:5, 'Ion_Density'])
date1 = dt.datetime(2020, 1, 1)
date2 = dt.datetime(2020, 1, 1, 1, 30, 30)
print(ivm[date1:date2, 'Ion_Density'])
print('')

In [ ]:
# Metadata
print('Metadata Information : Ion Density')
print(ivm.meta['Ion_Density'])
print('Ion Density units : ' + ivm.meta['Ion_Density', 'units'])

# Generalized metadata access that is independent of file standard
print('General technique : ')
print('Ion Density units : ' + ivm.meta['Ion_Density', ivm.meta.labels.units])
print('')

In [ ]:
# Full list of variables available from `ivm.variables`.
# Not that ICON variables are altered by pysat upon load to remove
# a preamble to make working with the variables easier.
print('Modified variable names : ')
print(ivm.variables[:5])
print('')

# The original names, as in the file, may be retained
ivm = pysat.Instrument('icon', 'ivm', inst_id='a', keep_original_names=True)
ivm.load(2020, 1)
print('Original variable names : ')
print(ivm.variables[:5])
print('')

In [ ]:
# Filenames found on local system
print('Local filenames located at `ivm.files.files`')
print('Subset of files : ')
print('by index')
print(ivm.files[0:3])
print('')
print('by date')
print(ivm.files[dt.datetime(2020, 1, 1):dt.datetime(2020, 1, 4)])

## IVM Basic Density Plot

In [ ]:
# Make basic time series plot of Ion Density
ivm = pysat.Instrument('icon', 'ivm', inst_id='a')
# load day of data
ivm.load(2020, 1)

# Basic plot of ion density
ivm['Ion_Density'].plot()
plt.title('ICON IVM Ion Density')
plt.ylabel('Ion Density (N/cc)')

# Basic plot of the log of ion density
plt.figure()
np.log10(ivm['Ion_Density']).plot()
plt.title('ICON IVM Log Ion Density')
plt.ylabel('Log Ion Density (N/cc)')

## IVM Per-Orbit Density Plots

In [ ]:
# Produce ion density plots orbit-by-orbit.
# While ICON products include an orbit number based upon the position of the
# spacecraft this breakdown does not produce a geophysical organization
# of the data.

# We demonstrate using pysat to parse the data orbit-by-orbit using Magnetic
# Local Time (MLT) values in the file to make orbits between 0-24 MLT.

# Set date for plotting IVM data
date = dt.datetime(2020, 1, 1)

# pysat is able to calculate orbit breaks in real-time.
# Setup parameters for pysat to use when determining orbits
orbit_info = {'kind': 'lt', 'index':'Magnetic_Local_Time'}
# 'kind' sets the type of orbit breakdown, lt for local time.
# 'index' sets the variable that will be used for determining orbit breaks.
# The expected period may also be set, defaults to a 97 minute LEO period.

# Instantiate ICON IVM Instrument object with orbit information
ivm = pysat.Instrument('icon', 'ivm', inst_id='a', clean_level='clean', 
                       orbit_info=orbit_info)

# Set the start and stop dates when iterating over Instrument object.
# By default, pysat will iterate over all available data.
# Two days chosen here to demonstrate orbits cross file and day boundaries.
ivm.bounds = (date, date + dt.timedelta(days=1))

# Iterate over the IVM data orbit-by-orbit. Each loop, a single orbit of
# data is available in ivm.data.
for livm in ivm.orbits:
    if livm.orbits.current == 0:
        # Only plot the first orbit of each day.
        plt.figure()
        np.log10(livm['Ion_Density']).plot()
        title = ''.join(['ICON IVM Ion Density {date:s} Orbit: {num:2d}',
                         '']).format(date=livm.date.strftime('%Y-%m-%d'),
                                     num=livm.orbits.current)
        plt.ylabel('Log Ion Density (N/cc)')



## Summary Plot - Per Orbit Data Breakdown

In [ ]:
# Summary plot orbit-by-orbit, single figure

# Set date for plotting IVM data
date = dt.datetime(2020, 1, 1)

# pysat is able to caculate orbit breaks in real-time.
# Setup parameters for pysat use
orbit_info = {'kind': 'lt', 'index':'Magnetic_Local_Time'}
# 'kind' sets the type of orbit breakdown, lt for local time.
# 'index' sets the variable that will be used for determining orbit breaks.

# Instantiate ICON IVM Instrument object
ivm = pysat.Instrument('icon', 'ivm', inst_id='a', clean_level='clean', 
                       orbit_info=orbit_info)

# Set the start and stop dates when iterating over Instrument object.
# By default, will iterate over all available data.
ivm.bounds = (date, date)

# Create custom method to add Log Ion Density to Instrument object
def add_log_density(inst):
    """Add the log of ion density to `inst`.
    
    Parameters
    ----------
    inst : pysat.Instrument
        ICON IVM pysat.Instrument object.
    """
    inst['Log_Ion_Density'] = np.log10(inst['Ion_Density'])
    return

# Attach custom function to Instrument object.
# This method is applied automatically to the IVM data every time new data is
# loaded. This feature allows a developer to customize the data in an Instrument
# object even when processed by third-party analysis modules.
ivm.custom_attach(add_log_density)

# Create a figure so each orbit is on the same plot.
fig, axs = plt.subplots(5, 1, figsize=(8.5,11))

# Use pysat functionality to iterate over the data, orbit by orbit.
# Each loop, a single orbit of data available in `ivm.data`.
for livm in ivm.orbits:
    # Log density plot
    livm.data.plot(x='Magnetic_Local_Time', y='Log_Ion_Density',
                   xlim=(0,24), xticks=[],
                   ax=axs[0], legend=False)
    axs[0].set_title('ICON IVM Log Ion Density')
    axs[0].set_ylabel('Log Ion Density (N/cc)')
    axs[0].set_xlabel('')
    
    # O+ fraction plot
    livm.data.plot(x='Magnetic_Local_Time', y='Fractional_Ion_Density_O',
                   xlim=(0,24), xticks=[],
                   ax=axs[1], legend=False)
    axs[1].set_title('ICON IVM O+ Fraction')
    axs[1].set_ylabel('O+ Fraction')
    axs[1].set_xlabel('')

    # Zonal Ion Velocity plot
    livm.data.plot(x='Magnetic_Local_Time', y='Ion_Velocity_Zonal',
                   xlim=(0,24), xticks=[],
                   ylim=(-300, 100),
                   ax=axs[2], legend=False)
    axs[2].set_title('ICON IVM Zonal Ion Velocity')
    axs[2].set_ylabel('Velocity (m/s)')
    axs[2].set_xlabel('')

    # Field-Aligned Velocity plot
    livm.data.plot(x='Magnetic_Local_Time', y='Ion_Velocity_Field_Aligned',
                   xlim=(0,24), xticks=[],
                   ylim=(-200, 200),
                   ax=axs[3], legend=False)
    axs[3].set_title('ICON IVM Field-Aligned Ion Velocity')
    axs[3].set_ylabel('Velocity (m/s)')
    axs[3].set_xlabel('')

    # Meridional Ion Velocity plot
    livm.data.plot(x='Magnetic_Local_Time', y='Ion_Velocity_Meridional',
                   xlim=(0,24), xticks=(0,6,12,18,24),
                   ylim=(-50, 50),
                   ax=axs[4], legend=False)
    axs[4].set_title('ICON IVM Meridional Ion Velocity')
    axs[4].set_ylabel('Velocity (m/s)')
    axs[4].set_xlabel('Magnetic Local Time')

    # For simplicity, only plot one orbit.
    break

# ICON MIGHTI Data Examples

In [ ]:
# First, obtain MIGHTI data from NASA CDAWeb.

# Instantiate pysat.Instrument objects for the MIGHTI data products
# MIGHTI Vector wind red
mighti_vw_red = pysat.Instrument('icon', 'mighti', tag='vector_wind_red', inst_id='vector')

# MIGHTI Vector wind green
mighti_vw_green = pysat.Instrument('icon', 'mighti', tag='vector_wind_green', inst_id='vector')

# MIGHTI Temperature
mighti_temp_a = pysat.Instrument('icon', 'mighti', tag='temperature',
                                 inst_id='a')
mighti_temp_b = pysat.Instrument('icon', 'mighti', tag='temperature',
                                 inst_id='b')

# collect into a list
mighti_insts = [mighti_vw_red, mighti_vw_green, mighti_temp_a, mighti_temp_b]

In [ ]:
# Download various MIGHTI data products

# If needed, change levels for logging printout to increase feedback.
# More information about logging may be found here:
# https://docs.python.org/3/library/logging.html
# pysat.logger.setLevel("INFO")

# Perform download for each dataset
for inst in mighti_insts:
    inst.download(start_download_date, stop_download_date)
    
# Change levels for logging printout to decrease feedback
# pysat.logger.setLevel("WARNING")

# MIGHTI Winds Basic Example

In [ ]:
# Produce simple overview plot of MIGHTI winds for a day of data.

# pysat features a variety of internal data checks to ensure data is
# consistent with expectations. Warnings are raised below relating to the
# treatment of ICON's metadata. An upcoming release to pysat improves the
# generality of metadata handling when loading files, which an upcoming
# release to pysatNASA utilizes. pysat > v3.0.1 and pysatNASA > 0.0.2

# Load day of data
mighti_vw_red.load(2020, 2)
mighti_vw_green.load(2020, 2)

# Zonal wind plot, red line
mighti_vw_red['Zonal_Wind'].transpose().plot()
plt.title('MIGHTI Zonal Wind')
plt.ylabel('Altitude (km)')

# Meridional wind plot, red line
plt.figure()
mighti_vw_red['Meridional_Wind'].transpose().plot()
plt.title('MIGHTI Meridional Wind')
plt.ylabel('Altitude (km)')

# Zonal wind plot, green line
plt.figure()
mighti_vw_green['Zonal_Wind'].transpose().plot()
plt.title('MIGHTI Zonal Wind')
plt.ylabel('Altitude (km)')

# Meridional wind plot, green line
plt.figure()
mighti_vw_green['Meridional_Wind'].transpose().plot()
plt.title('MIGHTI Meridional Wind')
plt.ylabel('Altitude (km)')

## MIGHTI Winds Summary Per-Orbit Example


In [ ]:
# Pick 'red' or 'green' for the line color MIGHTI is using
line_color = 'red'

# Set date
date = dt.datetime(2020, 1, 2)

# Set up orbit calculation parameters. Note that the 'average_local_time'
# variable is not available in the MIGHTI files, it will be added by pysat.
orbit_info = {'kind': 'lt', 'index': 'average_local_time'}

# The local time of the profiles varies as a function of height,
# make an average local time for use in calculations.
def add_average_local_solar_time(inst):
    """Adds mean profile local solar time as variable """

    # Average local time over profile.
    inst['average_local_time'] = inst['Local_Solar_Time'].mean(axis=1)

    # Remove overall times when there is no local time info.
    idx, = np.where(~inst['average_local_time'].isnull())
    inst.data = inst[idx]

    return
    
# MIGHTI Vector Wind
mighti_vw_color = pysat.Instrument('icon', 'mighti',
                                   tag='vector_wind_' + line_color,
                                   inst_id='vector',
                                   orbit_info=orbit_info)

# Confine orbit calculation to user set date
mighti_vw_color.bounds = (date, date)

# Add calculation of profile average local time
mighti_vw_color.custom_attach(add_average_local_solar_time)

# Create a figure so each orbit is on the same plot
# Zonal wind
fig, axs = plt.subplots(4, 1, figsize=(8.5,21))
# Meridional wind
figm, axsm = plt.subplots(4, 1, figsize=(8.5,21))

# Iterate over orbits and produce plots.
for i, lmighti_vw in enumerate(mighti_vw_color.orbits):
    # Enable support for plotting local_time on x-axis.
    alt = ("Epoch", lmighti_vw['average_local_time'].data)

    # Zonal wind.
    a = lmighti_vw['Zonal_Wind'].assign_coords(average_local_time=alt)

    # Make plot.
    a.transpose().plot(ax=axs[i], x='average_local_time', y='Altitude')
    axs[i].set_xlim((0, 24))
    axs[i].set_xticks([])
    axs[i].set_xlabel('')

    # Meridional wind.
    a = lmighti_vw['Meridional_Wind'].assign_coords(average_local_time=alt)

    # Make plot.
    a.transpose().plot(ax=axsm[i], x='average_local_time', y='Altitude')
    axsm[i].set_xlim((0, 24))
    axsm[i].set_xticks([])
    axsm[i].set_xlabel('')

    # For simplicity, terminate early.
    if i == 3:
        break


axs[0].set_title('MIGHTI Zonal Wind on ' + date.strftime('%Y-%m-%d'))
axs[-1].set_xticks([0, 6, 12, 18, 24])
axs[-1].set_xlabel('Profile Average Solar Local Time')

axsm[0].set_title('MIGHTI Meridional Wind on ' + date.strftime('%Y-%m-%d'))
axsm[-1].set_xticks([0, 6, 12, 18, 24])
axsm[-1].set_xlabel('Profile Average Solar Local Time')

## MIGHTI Temperature Example

In [ ]:
# pick 'a' or 'b' for the MIGHTI instrument
inst_id = 'a'

# Set date
date = dt.datetime(2020, 1, 4)

# Set parameters for orbit calculations.
# Note that 'average_local_time' is a custom variable.
orbit_info = {'kind': 'lt', 'index': 'average_local_time'}
    
# MIGHTI Temperature Instrument
mighti = pysat.Instrument('icon', 'mighti', tag='temperature',
                          inst_id=inst_id, orbit_info=orbit_info)

# Confine orbit calculation to user set date
mighti.bounds = (date, date)

# Produce a simpler local time signal to work with
def add_average_tangent_solar_time(inst):
    """Adds mean profile local solar time as variable """
    # Average local time over profile
    inst['average_local_time'] = inst['Tangent_Local_Solar_Time'].mean(dim='Alt')
    idx, = np.where(~inst['average_local_time'].isnull())
    inst.data = inst[idx]
    return

# Add custom method - applied whenever mighti data is loaded.
mighti.custom_attach(add_average_tangent_solar_time)

# Create figure
first_loop = True
for i, lmighti in enumerate(mighti.orbits):
    if first_loop:
        # Create figure on first loop. We now know how many orbits in the day.
        # fig, axs = plt.subplots(mighti.orbits.num, 1, figsize=(8.5,21))

        # For simplicity, limited to only three orbits.
        fig, axs = plt.subplots(3, 1, figsize=(8.5,21))
        first_loop = False
    
    # Enable support for plotting local_time on x-axis
    slt = ("Epoch", lmighti['average_local_time'].data)

    # Enable support for plotting tangent altitude on y axis
    alt = (("Epoch", "Alt"), lmighti['Tangent_Altitude'].data)

    # Apply coordinates
    a = lmighti['Temperature'].assign_coords(average_local_time=slt,
                                             Tangent_Altitude=alt)

    # Ensure local time is monotonically increasing
    a = a.sortby(lmighti['average_local_time'])

    # Make plot
    a.transpose().plot(ax=axs[i], x='average_local_time', y='Tangent_Altitude')
    axs[i].set_xlim((0, 24))
    axs[i].set_xticks([])
    axs[i].set_xlabel('')

    if i == 2:
        break

# Set title and outermost x axis label
axs[0].set_title('MIGHTI Temperature on ' + date.strftime('%Y-%m-%d'))
axs[-1].set_xticks([0, 6, 12, 18, 24])
axs[-1].set_xlabel('Profile Average Solar Local Time')

In [ ]:
mighti['Tangent_Altitude']

# ICON FUV Example


In [ ]:
# Download various FUV data products.

# By default the logger output is shown for the download below. While
# highlighted in red in binder, this feedback is informational,
# and not a warning.

# If needed, change levels for logging printout to increase feedback
# more information about logging may be found here
# https://docs.python.org/3/library/logging.html
pysat.logger.setLevel("INFO")

fuv_day = pysat.Instrument('icon', 'fuv', tag='day')
fuv_night = pysat.Instrument('icon', 'fuv', tag='night')

# Combine into list
fuv_insts = [fuv_day, fuv_night]

# Perform download for each dataset
for inst in fuv_insts:
    inst.download(start_download_date, stop_download_date)
    
# Change levels for logging printout to decrease feedback
pysat.logger.setLevel("WARNING")


In [ ]:
# Complete FUV Daytime data summary plot orbit-by-orbit

# Set date for the plot
date = dt.datetime(2020, 1, 4)

# pysat is able to calculate orbit breaks in real-time. Setup parameters for
# pysat use.
orbit_info = {'kind': 'lt', 'index': 'Local_Solar_Time_Disk'}
# 'kind' sets the type of orbit breakdown, lt for local time.
# 'index' sets the variable that will be used for determining orbit breaks.

# Instantiate ICON IVM Instrument object.
fuv = pysat.Instrument('icon', 'fuv', tag='day', clean_level='clean', 
                       orbit_info=orbit_info)
print(fuv.files.files)
# Filter out areas with no disk_ON2 values.
def filter_no_measurement_areas(inst):
    """Remove all measurements for times 'disk_ON2'==0"""
    inst.data = inst.data.where(inst['Disk_ON2'] > 0, drop=True)
    return

# Attach filter to object, run automatically whenever data is loaded.
fuv.custom_attach(filter_no_measurement_areas)

# Set the start and stop dates when iterating over Instrument object.
# By default, will iterate over all available data.
fuv.bounds = (date, date)

# Create a figure so each orbit is on the same plot.
fig, axs = plt.subplots(2, 1, figsize=(8.5,8))

for ax in axs:
    ax.set_xlim((6, 18))
    ax.set_xlabel('')
    ax.set_xticks([])
    
# use pysat functionality to iterate over the data, orbit by orbit.
# Each loop, a single orbit of data made available in `fuv.data`.
for lfuv in fuv.orbits:
    
    # disk_ON2
    axs[0].plot(lfuv['Local_Solar_Time_Disk'], lfuv['Disk_ON2'])
    axs[0].set_title('FUV O/N2 Ratio')
    axs[0].set_ylabel('O/N2 Ratio')
    
    # disk_sigma_ON2
    axs[1].plot(lfuv['Local_Solar_Time_Disk'], lfuv['Disk_Sigma_ON2'])
    axs[1].set_title('FUV O/N2 Uncertainty')
    axs[1].set_ylabel('O/N2 Ratio Standard Deviation')

    # For simplicity, only show one orbit
    break

axs[-1].set_xticks((6, 12, 18))
axs[-1].set_xlabel('Local Solar Time')
fig.tight_layout()

In [ ]:
# Complete FUV Nighttime summary plot orbit-by-orbit

# date for plot
date = dt.datetime(2020, 1, 4)

orbit_info = {'kind': 'lt', 'index': 'maximum_local_time'}
fuv = pysat.Instrument('icon', 'fuv', tag='night', clean_level='clean', 
                       orbit_info=orbit_info)

def add_max_stripe_local_time(inst):
    """Add maximum local time over stripes"""
    inst['maximum_local_time'] = inst['Local_Solar_Time'].max(axis=1)
    return

def filter_low_nmf2(inst, min_value=1.E2):
    """Filter out low NmF2 values"""
    inst['NMF2'] = inst['NMF2'].where(inst['NMF2'] > min_value)
    return

def shift_local_time(inst):
    """Shifts local times from 0-24 to 12-36"""
    idx, = np.where(inst['maximum_local_time'] < 12.)
    inst[idx, 'maximum_local_time'] += 24.
    return

def filter_o_profile(inst, min_value=1.E2):
    """Filter out low profile values"""
    inst['O_Plus_Density'] = inst['O_Plus_Density'].where(inst['O_Plus_Density'] > min_value)
    return

# Attach custom methods to object.
fuv.custom_attach(add_max_stripe_local_time)

# Note that applying custom keywords to these methods is supported.
fuv.custom_attach(filter_low_nmf2, kwargs={'min_value': 1.E1})
fuv.custom_attach(shift_local_time)
fuv.custom_attach(filter_o_profile)

# Set the start and stop dates when iterating over Instrument object.
fuv.bounds = (date, date)

# Create a figure for each product
fig, axs = plt.subplots(2, 1, figsize=(8.5,8))
# Create a larger plot for O+ density profiles
fig_o, axs_o = plt.subplots(4, 1, figsize=(8.5,18))

# Set some basic plot properties here
for ax in axs:
    ax.set_xlim((18, 30))
    ax.set_xlabel('')
    ax.set_xticks([])
    
# use pysat functionality to iterate over the data, orbit by orbit.
# Each loop, a single orbit of data found in ivm.data
for i, lfuv in enumerate(fuv.orbits):
    
    # average of HmF2 over FUV stripes
    axs[0].plot(lfuv['maximum_local_time'], lfuv['HMF2'].mean(axis=1))
    axs[0].set_title('FUV HmF2')
    axs[0].set_ylabel('Altitude (km)')
    
    # average of NmF2 over FUV stripes
    axs[1].plot(lfuv['maximum_local_time'], np.log10(lfuv['NMF2'].mean(axis=1)))
    axs[1].set_title('FUV NmF2')
    axs[1].set_ylabel('Log Density (N/cc)')
    
    # make nice plot of O+ profiles over altitude
    a = lfuv['O_Plus_Density'].mean(axis=2).transpose()
    # enable support for plotting local time on x axis
    slt = ("Epoch", lfuv['maximum_local_time'].data)
    # apply coordinates 
    a = a.assign_coords(maximum_local_time=slt)
    # sort things out
    a = a.sortby('maximum_local_time')
    # make plot
    np.log10(a).plot(x='maximum_local_time', y='Altitude', vmin=2., vmax=6.5,
                     ax=axs_o[i], xlim=(18, 30), xticks=[])
    axs_o[i].set_xlabel('')

    if i == 3:
        break
    
# labels for HmF2 and NmF2 plots
axs[-1].set_xticks((18, 24, 30))
axs[-1].set_xlabel('Local Solar Time')
fig.tight_layout()

# labels for O+ profile plots
axs_o[-1].set_xticks((18, 21, 24, 27, 30))
axs_o[-1].set_xlabel('Local Solar Time')
axs_o[0].set_title('O+ Profile Density')

# ICON EUV Data

In [ ]:
# Download EUV data products

# By default the logger output is shown for the download below. While
# highlighted in red in binder, this feedback is informational,
# and not a warning.

# if needed, change levels for logging printout to increase feedback
# more information about logging may be found here
# https://docs.python.org/3/library/logging.html
pysat.logger.setLevel("INFO")

# Instantiate EUV pysat.Instrument
euv = pysat.Instrument('icon', 'euv')

# Download
euv.download(start_download_date, stop_download_date)
    
# Change levels for logging printout to decrease feedback
pysat.logger.setLevel("WARNING")

In [ ]:
# complete EUV summary plot orbit-by-orbit

# set date for plot
date = dt.datetime(2020, 1, 1)


# instantiate ICON EUV Instrument object
orbit_info = {'kind': 'lt', 'index': 'Local_Solar_Time'}
euv = pysat.Instrument('icon', 'euv', clean_level='clean',
                       orbit_info=orbit_info)

# Set the start and stop dates when iterating over Instrument object.
euv.bounds = (date, date)

# Create a figure for each product
fig, axs = plt.subplots(2, 1, figsize=(8.5,8))

# Create a larger plot for O+ density profiles
fig_o, axs_o = plt.subplots(4, 1, figsize=(8.5,18))

# Set some basic plot properties here
for ax in axs:
    ax.set_xlim((6, 18))
    ax.set_xlabel('')
    ax.set_xticks([])
    
# use pysat functionality to iterate over the data, orbit by orbit.
for i, leuv in enumerate(euv.orbits):
    
    # Average of HmF2 over FUV stripes
    axs[0].plot(leuv['Local_Solar_Time'], leuv['HmF2'])
    axs[0].set_title('EUV HmF2')
    axs[0].set_ylabel('Altitude (km)')
    
    # Average of NmF2 over FUV stripes
    axs[1].plot(leuv['Local_Solar_Time'], np.log10(leuv['NmF2']))
    axs[1].set_title('EUV NmF2')
    axs[1].set_ylabel('Log Density (N/cc)')
    
    # Make nice plot of O+ profiles over altitude
    a = leuv['Oplus']

    # Enable support for plotting local time on x axis
    slt = ("Epoch", euv['Local_Solar_Time'].data)

    # Apply coordinates
    a = a.assign_coords(Local_Solar_Time=slt)

    # Sort
    a = a.sortby('Local_Solar_Time')

    # Make plot
    np.log10(a).plot(x='Local_Solar_Time', y='Alt', vmin=2., vmax=6.5,
                     ax=axs_o[i], xlim=(6, 18), xticks=[])
    axs_o[i].set_xlabel('')

    if i == 3:
        break

# Labels for HmF2 and NmF2 plots.
axs[-1].set_xticks((6, 9, 12, 15, 18))
axs[-1].set_xlabel('Local Solar Time')
fig.tight_layout()

# Labels for O+ profile plots.
axs_o[-1].set_xticks((6, 9, 12, 15, 18))
axs_o[-1].set_xlabel('Local Solar Time')
axs_o[0].set_title('ICON EUV O+ Density Profile')